In [7]:
import numpy
import pandas as pd
import torch
import numpy as np
#from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import os,random
import torchvision.transforms as T
import csv
import cv2
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.autograd import Variable

#my imports
import mynet
import train

from torch.utils.tensorboard import SummaryWriter #tensorboard --logdir log --bind_all


ModuleNotFoundError: No module named 'mynet'

In [10]:
# 获取当前路径
current_directory = os.getcwd()
print("当前路径:", current_directory)

# 定义要变更的目标路径
target_directory = "D:/AI/boltpd"

# 使用os.chdir()来变更路径
os.chdir(target_directory)

# 再次获取当前路径，此时它应该是目标路径
new_current_directory = os.getcwd()
print("变更后的路径:", new_current_directory)


当前路径: /home/br-python/Documents/chen/20230925-test-CG


FileNotFoundError: [Errno 2] No such file or directory: 'D:/AI/boltpd'

In [6]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.tensor([1, 2, 3]).to(device)
print(x.device)

2.0.0+cu117
True


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def generate_dataset(dir):
    """
    set_label should be 'torch.tensor([1])' if two-catogory and positive sample
    """
    pathDir = os.listdir(dir)    #取图片的原始路径
    filenumber=len(pathDir)
    rate=1    #自定义抽取图片的比例，比方说100张抽10张，那就是0.1
    picknumber=int(filenumber*rate) #按照rate比例从文件夹中取一定数量图片
    sample = random.sample(pathDir, picknumber)  #随机选取picknumber数量的样本图片

    file = pd.read_csv(r'./label/label-2024pretension.csv')
    file=np.array(file)
    labels=[]
    img_data = []
    f0 = None
    for file_name in sample:
        if f0 is None:
            f0 = os.path.join(dir, file_name)
        if file_name != "Thumbs.db":
            if int(file_name[-9:-4]) > 9900 or int(file_name[-9:-4]) < 106:
                img_dir = os.path.join(dir, file_name)
                img = cv2.imread(img_dir)
                #ref_img = cv2.imread('./database/c_data/bolt1/DSC01686.JPG')
                #img = cv2.subtract(img, ref_img)
                #img1 = cv2.subtract(img,cv2.imread(f0))
                img = cv2.resize(img, (800, 800))   # /5 resize img
                #img_gray = cv.cvtColor(img,cv.COLOR_RniGB2GRAY)
                pimg = Image.fromarray(img)
                img_data.append(pimg)
                for item in file:       #add label, from 1row
                    sh = item[1]
                    if file_name[-9:-4] == sh[-9:-4]:
                        labels.append(item[0]/230)#
    labels = torch.tensor(labels)
    labels = labels.type(torch.FloatTensor)
    labels = labels.unsqueeze(1)
    return img_data,labels